In [156]:
from typing import List
from collections import deque

class Solution:
    def matrixRankTransform(self, matrix: List[List[int]]) -> List[List[int]]:
        def find(x):
            if father[x] != x:
                father[x] = find(father[x])
            return father[x]
        
        def union(x, y):
            x, y = find(x), find(y)
            if x < y:
                father[x] = y
            else:
                father[y] = x
        
        rows, cols = len(matrix), len(matrix[0])
        father = [i for i in range(rows * cols)] # 每个坐标点的父节点
        indeg = [0 for _ in range(rows * cols)] # 每个坐标点的入度
        nexts = [[] for _ in range(rows * cols)] # 比当前点大的值的位置
        for i in range(rows): # 每行的大小关系
            nums = []
            for j in range(cols):
                nums.append((matrix[i][j], i*cols+j))
            nums.sort()
            for j in range(1, cols):
                pre, nxt = nums[j-1], nums[j]
                if nxt[0] > pre[0]: # 经过排序之后，相邻两点 >、 ==
                    nexts[pre[1]].append(nxt[1])
                    indeg[nxt[1]] += 1
                else: # nums[j] == nums[j-1]，等于的情况，要uoinon一起，并将indeg算在 父节点
                    union(pre[1], nxt[1])
        
        for j in range(cols): # 每列的大小关系
            nums = []
            for i in range(rows):
                nums.append((matrix[i][j], i*cols+j))
            nums.sort()
            for i in range(1, rows):
                pre, nxt = nums[i-1], nums[i]
                if nxt[0] > pre[0]:
                    nexts[pre[1]].append(nxt[1])
                    indeg[nxt[1]] += 1
                else:
                    union(pre[1], nxt[1])
        
        # group, 每个点是否有与自己相同值的点
        group = [[] for _ in range(rows * cols)]
        for i in range(rows):
            for j in range(cols):
                cord = i * cols + j
                root = find(cord)
                group[root].append(cord)
                if father[cord] != cord:
                    indeg[root] += indeg[cord]
        
        # 找出入度为0的起始点
        points = deque()
        for i in range(rows):
            for j in range(cols):
                cord = i * cols + j
                if indeg[father[cord]] == 0:
                    points.append(cord)
                    
        rank = 1
        res = [[0] * cols for _ in range(rows)]
        while points:
            for _ in range(len(points)):
                cur = points.popleft()
                for member in group[cur]: # 把属于当前group的置为相同的rank
                    res[member//cols][member%cols] = rank
                
                    for nxt in nexts[member]: # 当前节点的 next节点的入度-1
                        indeg[father[nxt]] -= 1
                        if indeg[father[nxt]] == 0:
                            points.append(father[nxt])
            rank += 1
        return res

In [160]:
from typing import List
from collections import deque

class Solution:
    def matrixRankTransform(self, matrix: List[List[int]]) -> List[List[int]]:
        def find(x):
            if father[x] != x:
                father[x] = find(father[x])
            return father[x]
        
        def union(x, y):
            x, y = find(x), find(y)
            if x < y:
                father[x] = y
            else:
                father[y] = x
        
        rows, cols = len(matrix), len(matrix[0])
        father = [i for i in range(rows * cols)] # 每个坐标点的父节点
        indeg = [0 for _ in range(rows * cols)] # 每个坐标点的入度
        nexts = [[] for _ in range(rows * cols)] # 比当前点大的值的位置
        for i in range(rows): # 每行的大小关系
            nums = []
            for j in range(cols):
                nums.append((matrix[i][j], i*cols+j))
            nums.sort()
            for j in range(1, cols):
                pre, nxt = nums[j-1], nums[j]
                if nxt[0] > pre[0]: # 经过排序之后，相邻两点 >、 ==
                    nexts[pre[1]].append(nxt[1])
                    indeg[nxt[1]] += 1
                else: # nums[j] == nums[j-1]，等于的情况，要uoinon一起，并将indeg算在 父节点
                    union(pre[1], nxt[1])
        
        for j in range(cols): # 每列的大小关系
            nums = []
            for i in range(rows):
                nums.append((matrix[i][j], i*cols+j))
            nums.sort()
            for i in range(1, rows):
                pre, nxt = nums[i-1], nums[i]
                if nxt[0] > pre[0]:
                    nexts[pre[1]].append(nxt[1])
                    indeg[nxt[1]] += 1
                else:
                    union(pre[1], nxt[1])
        
        # group, 每个点是否有与自己相同值的点
        group = [[] for _ in range(rows * cols)]
        for i in range(rows):
            for j in range(cols):
                cord = i * cols + j
                root = find(cord)
                group[root].append(cord)
                if father[cord] != cord:
                    indeg[root] += indeg[cord]
        
        # 找出入度为0的起始点
        points = deque()
        for i in range(rows):
            for j in range(cols):
                cord = i * cols + j
                root = find(cord)
                if indeg[root] == 0:
                    points.append(cord)
                    
        rank = 1
        res = [[0] * cols for _ in range(rows)]
        while points: # points存的是父节点
            for _ in range(len(points)):
                cur = points.popleft()
                for member in group[cur]: # 把属于当前group的置为相同的rank
                    res[member//cols][member%cols] = rank
                
                    for nxt in nexts[member]: # 当前节点的 next节点的入度-1
                        root = find(nxt)
                        indeg[root] -= 1
                        if indeg[root] == 0:
                            points.append(root)
            rank += 1
        return res

### 方法2
    按照 matrix中元素的大小进行排序，在为元素进行赋值的时候，假设当前元素的坐标为（i,j）
    当前元素的rank要小于 i行的rank值，大于j列rank的值

In [186]:
from typing import List
from collections import deque, defaultdict

class Solution:
    def matrixRankTransform(self, matrix: List[List[int]]) -> List[List[int]]:
        def find(x):
            if father[x] != x:
                father[x] = find(father[x])
            return father[x]
        
        def union(x, y):
            x, y = find(x), find(y)
            if x < y:
                father[y] = x
            else:
                father[x] = y
                
        rows, cols = len(matrix), len(matrix[0])
        father = [x for x in range(rows*cols)]
        for i in range(rows):
            count = {}
            for j in range(cols):
                cord = i*cols+j
                if matrix[i][j] not in count:
                    count[matrix[i][j]] = cord
                else:
                    union(count[matrix[i][j]], cord)
                
        for j in range(cols):
            count = {}
            for i in range(rows):
                cord = i*cols+j
                if matrix[i][j] not in count:
                    count[matrix[i][j]] = cord
                else:
                    union(count[matrix[i][j]], cord)
        
        group = [[] for _ in range(rows * cols)]
        nums = []
        for i in range(rows):
            for j in range(cols):
                cord = i * cols + j
                root = find(cord)
                group[root].append(cord)
                nums.append((matrix[i][j], cord))
        
        # 按照大小依次
        nums.sort()
        res = [[0] * cols for _ in range(rows)]
        row_ranks = [0] * rows
        col_ranks = [0] * cols
        for val, cord in nums:
            r, c = cord//cols, cord%cols
            if res[r][c] != 0:
                continue
            root = find(cord)
            rank = 0
            for member in group[root]:
                m_r, m_c = member//cols, member%cols
                rank = max(rank, row_ranks[m_r])
                rank = max(rank, col_ranks[m_c])
            for member in group[root]:
                m_r, m_c = member//cols, member%cols
                res[m_r][m_c] = rank + 1
                row_ranks[m_r] = rank + 1
                col_ranks[m_c] = rank + 1
        return res

In [187]:
solution = Solution()
solution.matrixRankTransform([[-32,15,38,17,-44,43,42,-47,-44,-41],
                              [34,-43,-24,7,-10,-43,36,-5,-22,37],
                              [4,-13,-38,49,16,-21,30,13,-20,47],
                              [2,-35,32,11,26,-31,40,31,-46,-7],
                              [4,19,18,-27,16,43,-10,-11,44,39],
                              [18,9,48,-29,30,5,8,-13,-42,-43],
                              [48,47,30,29,24,-29,22,-31,12,-37],
                              [38,-23,44,-13,-46,37,-12,31,14,-31],
                              [-28,23,-50,-23,12,23,18,-11,-44,31],
                              [-10,37,16,11,-18,17,40,-41,26,-31]])

[[4, 12, 17, 13, 2, 19, 18, 1, 2, 3],
 [13, 1, 3, 9, 7, 1, 14, 8, 4, 17],
 [10, 9, 2, 20, 12, 7, 13, 11, 8, 19],
 [8, 2, 16, 10, 14, 3, 17, 15, 1, 6],
 [10, 14, 13, 5, 12, 19, 9, 7, 20, 18],
 [12, 11, 19, 4, 15, 8, 10, 6, 3, 1],
 [18, 17, 15, 14, 13, 6, 12, 5, 9, 4],
 [17, 6, 18, 7, 1, 16, 8, 15, 10, 5],
 [5, 15, 1, 6, 8, 15, 11, 7, 2, 16],
 [7, 16, 11, 10, 6, 12, 17, 2, 13, 5]]

In [148]:
[[4,12,17,13,2,19,18,1,2,3],
 [13,1,3,9,7,1,14,8,4,17],
 [10,9,2,20,12,7,13,11,8,19],
 [8,2,16,10,14,3,17,15,1,6],
 [10,14,13,5,12,19,9,7,20,18],
 [12,11,19,4,15,8,10,6,3,1],
 [18,17,15,14,13,6,12,5,9,4],
 [17,6,18,7,1,16,8,15,10,5],
 [5,15,1,6,8,15,11,7,2,16],
 [7,16,11,10,6,12,17,2,13,5]]

[[4, 12, 17, 13, 2, 19, 18, 1, 2, 3],
 [13, 1, 3, 9, 7, 1, 14, 8, 4, 17],
 [10, 9, 2, 20, 12, 7, 13, 11, 8, 19],
 [8, 2, 16, 10, 14, 3, 17, 15, 1, 6],
 [10, 14, 13, 5, 12, 19, 9, 7, 20, 18],
 [12, 11, 19, 4, 15, 8, 10, 6, 3, 1],
 [18, 17, 15, 14, 13, 6, 12, 5, 9, 4],
 [17, 6, 18, 7, 1, 16, 8, 15, 10, 5],
 [5, 15, 1, 6, 8, 15, 11, 7, 2, 16],
 [7, 16, 11, 10, 6, 12, 17, 2, 13, 5]]

In [149]:
[[4,2,3],
 [1,3,4],
 [5,1,6],
 [1,3,4]]

[[4, 2, 3], [1, 3, 4], [5, 1, 6], [1, 3, 4]]